In [1]:
import pandas as pd
import pickle
import numpy as np
import networkx as nx
import re
import time

In [23]:
df_ivs = pickle.load(open('data/ivs_exploded_100.p', 'rb'))
df_ivs_old = pickle.load(open('data/ivs_exploded_100_old.p', 'rb'))
routes_dests = pickle.load(open('data/users_ship_specific_routes.p', 'rb'))
df_ships = pd.read_excel('data/ship_types.xlsx')

G = pickle.load(open("data/network.p", "rb"))
paths = pickle.load(open("data/paths.p", "rb"))
df_random = pickle.load(open("data/df_random.p", "rb"))
df_abm = pickle.load(open("data/df_abm.p", "rb"))
non_zero_flows = pickle.load(open("data/non_zero_flows.p", "rb"))
feasible_combinations = pickle.load(open('data/feasible_comb.p', 'rb'))
optimal_flows = pickle.load(open('data/optimal_flows.p', 'rb'))
agent_data = pickle.load(open('data/agent_data.p', 'rb'))
model_data = pickle.load(open('data/model_data.p', 'rb'))
path_lengths = pickle.load(open('data/path_lengths_ship_specific_routes.p', 'rb'))

In [24]:
a = pd.DataFrame.from_dict(model_data.loc[1440].item()).sort_values(by='id')
# b = a.loc[a.time_departed>(24*60)]
a['pure_travel_time'] = a['travel_time'] - a['time_in_line'] - a['time_charging']
b = a.loc[a.route==('NLGOR', 'NLRTM', 0)]

In [25]:
a

,id,route,time_departed,travel_time,time_in_line,time_charging,time_charging_dest,full_charging_info,distance_travelled,battery_size,pure_travel_time
1,0,"(NLMOE, NLOOS, 0)",4,148,0,0,0,{},37094.757687,1450,148
2,1,"(NLMOE, NLOOS, 0)",22,148,0,0,0,{},37094.757687,0,148
7,2,"(NLNRW, NLBON, 0)",40,188,0,0,0,{},47022.905287,1000,188
32,3,"(NLTLB, NLRTM, 1)",41,574,0,162,0,"{'8863818': 73, '8862663': 89}",103223.178110,4750,412
15,4,"(NLTLB, NLMOE, 0)",42,375,51,23,23,{'8863818': 23},68889.728473,1450,301
...,...,...,...,...,...,...,...,...,...,...,...
103,112,"(NLNIJ, NLDRU, 0)",1261,143,0,0,0,{},35931.820139,4750,143
105,113,"(NLMOE, NLOOS, 0)",1266,148,0,0,0,{},37094.757687,4750,148
102,115,"(NLTNZ, NLBRS, 0)",1277,121,0,0,0,{},30324.711900,4750,121
108,116,"(NLMOE, NLOOS, 0)",1283,148,0,0,0,{},37094.757687,4750,148


## Try with agent_data

In [26]:
c = agent_data.reset_index()

In [28]:
c.loc[c.AgentID==49].groupby('vessel_status').count()

,Step,AgentID,id,vessel_route,battery_size,battery_level,generated_at,removed_at,station_status
vessel_status,,,,,,,,,
charging,236,236,236,236,236,236,236,0,0
driving,314,314,314,314,314,314,314,0,0
inline,265,265,265,265,265,265,265,0,0


In [ ]:
c.loc[c.AgentID == 13].groupby('vessel_status').count()

In [ ]:
c.loc[c.AgentID == 77].groupby('vessel_status').count()

In [ ]:
c.loc[c.AgentID == 1]

In [ ]:
a = agent_data.loc[agent_data.station_status==1].reset_index().sort_values(by='Step')
a.groupby(['AgentID']).count()

In [ ]:
non_zero_flows[('NLRTM', 'NLMOE', 0)]

In [7]:
paths[('NLRTM', 'NLMOE', 0)]

['8863306',
 '8864009',
 '8863742',
 '8867500',
 '8865140',
 '8866686',
 '8863048',
 '8866570',
 '8863131',
 '8865003',
 '8863665',
 '8862663',
 '8860725',
 '30984595']

In [14]:
b = paths[('NLGOR', 'NLRTM', 0)]

In [15]:
b = b[::-1]

In [16]:
for i in range(1):
    print(i)

0


In [17]:
a1 =0
for i in range(len(b)-1):
    a = nx.dijkstra_path_length(G, b[i], b[i+1], weight='length_m')
    a1 += a
    print(a, a1)

651.3288639667403 651.3288639667403
6345.434042802258 6996.762906768998
3010.574501261619 10007.337408030617
4548.7114559023 14556.048863932916
1910.6073697711045 16466.65623370402
4354.183792219703 20820.840025923724
2859.5103864259836 23680.350412349708
831.6178353275283 24511.968247677236
584.9264533641734 25096.89470104141
24037.539243228617 49134.43394427003
7912.849145725791 57047.28308999582
1655.7769099938282 58703.05999998965
690.3248762319221 59393.38487622157


In [ ]:
a1

In [18]:
path_lengths[('NLRTM', 'NLAPN', 0)]

46113.84265717132

In [19]:
path_lengths[('NLTNZ', 'NLVLI', 0)]

27535.141001402484

In [ ]:
paths[('NLKGZ', 'NLAMS', 0)]

In [ ]:
agent_data.reset_index().loc[agent_data.reset_index().AgentID.str.len()<4]

In [ ]:
agent_data.reset_index(inplace=True)

In [ ]:

agent_data.loc[agent_data.station_status == 1]

In [69]:
agent_data.loc[agent_data.Vessel_status!=0]

AttributeError: 'DataFrame' object has no attribute 'Vessel_status'

In [ ]:
df_abm.loc[df_abm.name]

In [ ]:
o = {}
for key, item in optimal_flows.items():
    if item != 0:
        a = (re.sub('''["'_']''', "", key[16:35]).split(','))
        a = tuple([a[0], a[1], int(a[2])])
        o[a] = {'combinations':[], 'flows':[]}

for key, item in optimal_flows.items():
    if item != 0:
        a = (re.sub('''["'_']''', "", key[16:35]).split(','))
        a = tuple([a[0], a[1], int(a[2])])
        o[a]['combinations'].append(key[40:-4].split("',_'"))
        o[a]['flows'].append(item)
        print(a)
        print(key[40:-4].split("',_'"))


In [ ]:
o

In [ ]:
a = o[('NLAMS', 'NLUTC', 1)]
combi = np.random.choice(a=np.arange(len(a)), size=1, replace=False, p=[0.5,0.5])

In [ ]:
a = [1,2,3]
a = [i/sum(a) for i in a]

In [ ]:
a

In [ ]:
df_abm.loc[df_abm.name == '8864005']

In [ ]:
updated_feasible_combinations = {}
for key, combinations in feasible_combinations.items():
    updated_feasible_combinations[key] = []
    if len(combinations) > 1:
        for combi in combinations:
            # print(combi)
            feasible = True
            for node in combi:
                if not node in charging_points:
                    feasible = False
                    break
            if feasible:
                updated_feasible_combinations[key].append(combi)
    else:
        feasible = True
        for node in combinations:
            if not node in charging_points:
                feasible = False
                break
        if feasible:
            updated_feasible_combinations[key].append(combinations)



In [ ]:
a = [1]
a[0]

In [ ]:
# extract data
df_links = nx.to_pandas_edgelist(G)
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [ ]:
for _, row in df_abm.iterrows():  # index, row in ...
    # create agents according to model_type

    model_type = row['model_type']
    agent = None

    name = row['name']
    if pd.isna(name):
        name = ""
        print("error, nameless entry")
    else:
        name = name
    break

In [ ]:
a =[1,3,34,6]
a[::-1]

In [ ]:
df_random.trip_count.iloc[0]

In [ ]:
%%time
type_list = list(df_random.iloc[:, 4:-2])
for hour in range(24):
    df_1 = df_random.loc[(df_random.hour == hour)]
    for timestep in range(60):
        for harbour in list(set(list(df_1.origin.unique()) + list(df_1.destination.unique()))):
            a = df_1.loc[(df_1.origin==harbour)|(df_1.destination == harbour)]
            for i,j in enumerate(a.index):
                # chance that a vessel is generated is equal to 1/2 (either spawn at origin or dest) * trip_count/365
                # (because trip_count is yearly) *(time_step/hours)
                if 0.5 * (df_1.trip_count[j]/365) * (1/60) >= np.random.random():
                    # determine vessel type
                    prob = list(a.iloc[i, 4:-2].values / a.iloc[i, 4:-2].sum())
                    to_pick = type_list
                    ship_type = np.random.choice(a=to_pick, size=1, replace=True, p=prob)
                    # randomly draw ship types for each OD pair
                    print(ship_type, "Vessel departed at", df_1.origin[j], hour, ':', timestep, "heading to", df_1.destination[j], "via route", df_1.key[j])


In [ ]:
type_list

In [ ]:
def random_vessel_generator(df_prob, load=1):
    """ Function to generate random vessel data.
    Parameters
    ----------
    df_prob: pd.DataFrame
    This dataframe must contain historical travel data
    load: float
    This float is equal to the simulated occupation of the network and may be read as the part of the traffic that is
    electric.
    """
    df_prob = df_prob.loc[df_prob.trip_count != 0]
    df_prob.reset_index(inplace=True, drop=True)
    df_prob = df_prob.fillna(0)

    # create dict to store random prob based values later on
    main_dict = {i: [] for i in df_prob.columns}

    # copy origin and destination from original df
    main_dict['origin'] = []
    main_dict['destination'] = []
    main_dict['route_v'] = []
    main_dict['hour'] = []
    main_dict['trip_count'] = []

    # fill nan values and create type list
    type_list = list(df_prob.iloc[:, 4:-1])
    for i in range(24):
        df_temp = df_prob.loc[df_prob.hour == i]
        df_temp = df_temp.reset_index(drop=True)
        total = df_temp.trip_count.sum()
        prob_r = [i / total for i in df_temp.trip_count]
        to_pick_r = list(np.arange(0, len(df_temp)))
        count_r = total * 1
        # randomly draw ODs
        rand_routes = np.random.choice(a=to_pick_r, size=round(count_r), replace=True, p=prob_r)
        unique_r, counts_r = np.unique(rand_routes, return_counts=True)
        temp_type_dict_r = dict(zip(unique_r, counts_r))

        for pair in df_temp.index:
            # store trip count data previous draw
            if pair in temp_type_dict_r.keys():
                main_dict['trip_count'].append(temp_type_dict_r[pair])
                main_dict['origin'].append(df_temp.origin[pair])
                main_dict['destination'].append(df_temp.destination[pair])
                main_dict['route_v'].append(df_temp.route_v[pair])
                main_dict['hour'].append(i)
                # print(main_dict['trip_count'])
                # determine parameters to randomly draw ship types for each OD pair
                prob = list(df_temp.iloc[pair, 4:-1].values / df_temp.iloc[pair, 4:-1].sum())
                to_pick = type_list
                # goes wrong now because may be zero ...
                count = temp_type_dict_r[pair]

                # randomly draw ship types for each OD pair
                rand_vessels = np.random.choice(a=to_pick, size=round(count), replace=True, p=prob)
                unique, counts = np.unique(rand_vessels, return_counts=True)
                temp_type_dict = dict(zip(unique, counts))
                for key in type_list:
                    if key in temp_type_dict.keys():
                        main_dict[key].append(temp_type_dict[key])
                    else:
                        main_dict[key].append(0)
    break

    df_return = pd.DataFrame.from_dict(main_dict)

    return df_return

In [ ]:
a1 = random_vessel_generator(df_ivs, 1)

In [ ]:
a1